# Download Data

In [4]:
from utils.util import load_class_names,convert_class_names_to_idx
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load class names
class_names_loaded = load_class_names("data/class_names_animals.txt") 
class_name_to_idx = convert_class_names_to_idx(class_names_loaded)

In [5]:
from pathlib import Path
data_dir = Path("data")
if data_dir.exists():
    pass
else:
    # If the file doesn't exist, create it
    data_dir.mkdir(parents=True,
                   exist_ok=True)

In [6]:
from utils.util import download_data
for animal in class_names_loaded:
    download_path = data_dir / str(animal + ".npy")
    if download_path.exists():
        pass
    else:
        download_data(animal)

# Create the model

In [7]:
from utils.model import create_mobilenetv3_model
mobileNetV3, mobileNetV3_transform = create_mobilenetv3_model()

In [8]:
from torchinfo import summary

summary(mobileNetV3,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
MobileNetV3 (MobileNetV3)                                    [32, 3, 224, 224]    [32, 42]             --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 960, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 16, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 16, 112, 112]   (432)                False
│    │    └─BatchNorm2d (1)                                  [32, 16, 112, 112]   [32, 16, 112, 112]   (32)                 False
│    │    └─Hardswish (2)                                    [32, 16, 112, 112]   [32, 16, 112, 112]   --                   --
│    └─InvertedResidual (1)                                  [32, 16, 112, 112]   [32, 

# Create dataloaders

In [9]:
from torchvision import transforms
from utils.data_setup import get_train_test_per_animal

training_transforms = transforms.Compose([transforms.TrivialAugmentWide(),mobileNetV3_transform])

train_data, test_data = [],[]
for animal in class_names_loaded:
    train_animal, test_animal = get_train_test_per_animal(animal,class_name_to_idx)
    train_data.extend(train_animal)
    test_data.extend(test_animal)

In [10]:
from utils.data_setup import create_dataloaders
BATCH_SIZE = 32
train_dataloader, test_dataloader = create_dataloaders(
    train_data = train_data,
    test_data = test_data,
    train_transforms = training_transforms,
    test_transforms = mobileNetV3_transform,
    batch_size = BATCH_SIZE,
)

# Training the model

In [11]:
import torch
from utils import engine

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing = 0.1)
optimizer = torch.optim.Adam(params = mobileNetV3.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=5)
early_stopping = engine.EarlyStopping(tolerance=5, min_delta=10)

In [12]:
from utils import engine
EPOCHS = 1

results = engine.train(model=mobileNetV3,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      epochs=EPOCHS,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      scheduler = scheduler,
                      device=device)

  0%|          | 0/1 [00:00<?, ?it/s]

# Saving Model

In [ ]:
from utils.util import save_model
save_model(model = mobileNetV3,
                   target_dir = "models",
                   model_name = f"mobileNetV3_quickdraw_animals_epoch_{EPOCHS}.pth")